# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Trương Gia Tiến
- MSSV: 20127643
- Lớp: 20CLC02

# Import

In [2258]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2259]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_train_clone = X_train.copy().to_numpy()
y_train_clone = y_train.copy().to_numpy()

X_test_clone = X_test.copy().to_numpy()
y_test_clone = y_test.copy().to_numpy()
# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [2260]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)  

def mse(y, y_hat):
    return np.mean((y.ravel() - y_hat.ravel())**2)


# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [2261]:
# Phần code cho yêu cầu 1a
lr = OLSLinearRegression()
lr.fit(X_train, y_train)

coef = lr.get_params()

y_pred = lr.predict(X_test)
print("X train: ", X_train.shape)
print("Y train: ", y_train.shape)

print("Y predict: ", y_pred.shape)

# print("Coef - w: \n", coef)


X train:  (1085, 10)
Y train:  (1085,)
Y predict:  (95,)


In [2262]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
rmse = np.sqrt(mse(y_test, y_pred))

print("RMSE = ",rmse)

RMSE =  7.064046430584034


Công thức hồi quy

$$\text{Life expectancy} = 0.015101 * AdultMortality + 0.090220 * BMI + 0.042922 * Polio + 0.139289 * Diphtheria$$ 
$$\text+ (-0.567333) * HIV/AIDS + (-0.000101) * GDP + 0.740713 * thinnessAge10\_19$$ 
$$\text+ 0.190936 * ThinnessAge5\_9 + 24.505974 * IncomCompositionOfResources$$
$$\text+ 2.393517 * Schooling$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [2263]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

kf = KFold(n_splits=5, shuffle=True, random_state=None)
cross_validation_RMSE_list = []
for i in range(10):
    X_train_feature = X_train_clone[:,i].copy()
    y_train_feature = y_train_clone.copy()
    
    RMSE = 0
    # for j in range(5):
    for trainIndex, testIndex in kf.split(X_train_feature, y_train_feature):
        X_train_kFold, X_val = X_train_feature[trainIndex].reshape(868,1), X_train_feature[testIndex].reshape(217,1)
        y_train_kFold, y_val = y_train_feature[trainIndex].reshape(868,1), y_train_feature[testIndex].reshape(217,1)

        lr = OLSLinearRegression()
        lr.fit(X_train_kFold, y_train_kFold)
        y_val_pred = lr.predict(X_val)
        RMSE += np.sqrt(mse(y_val, y_val_pred))
    cross_validation_RMSE_list.append(RMSE / 5)

    # break
#     sum = 0
#     for j in range(5):
#         X_val = X_train_feature[:,j].reshape(217,1)
#         y_val = y_train_feature[:,j].reshape(217,1)

#         temp = X_train_feature.copy()
#         temp = np.delete(temp, j, 1)
#         X_train_kFold = temp.copy().reshape(217 * 4,1)

#         temp = y_train_feature.copy()
#         temp = np.delete(temp, j, 1)
#         y_train_kFold = temp.copy().reshape(217 * 4)

#         lr = OLSLinearRegression()
#         lr.fit(X_train_kFold, y_train_kFold)
#         y_val_pred = lr.predict(X_val)

#         sum += np.sqrt(mse(y_val, y_val_pred))
#     RMSE_shuff.append(sum / 5)


RMSE_shuff = np.asarray(cross_validation_RMSE_list) # chuyen thanh numpy array
bestModelIndex = np.argmin(cross_validation_RMSE_list) # index cua min RMSE
feature = ["Adult Mortality", "BMI", "Polio", "Diphtheria", "HIV/AIDS", "GDP", "Thinness age 10-19", "Thinness age 5-9", "Income composition of resources", "Schooling"]

print("Cross validations: ", cross_validation_RMSE_list)
print("\nBest feature:", feature[bestModelIndex], "\nBest model's cross-validation value = ", cross_validation_RMSE_list[bestModelIndex])

Cross validations:  [46.151148817625774, 27.907380619963114, 18.045593563981164, 16.050239108204696, 67.13923458569373, 60.20577569061152, 51.84727353661312, 51.799388252684665, 13.223384843165036, 11.783660475741138]

Best feature: Schooling 
Best model's cross-validation value =  11.783660475741138


In [2264]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
lr = OLSLinearRegression()

lr.fit(X_train_clone[:,bestModelIndex].reshape(1085,1),y_train)
y_test_ch_pred = lr.predict(X_test_clone[:,bestModelIndex].reshape(95,1))

In [2265]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
rmse_bestModel = np.sqrt(mse(y_test, y_test_ch_pred))
print(rmse_bestModel)

10.26095039165537


Công thức hồi quy

$$\text{Life expectancy} = ...$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [2266]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [2267]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [2268]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy

$$\text{Life expectancy} = ...$$